<a href="https://colab.research.google.com/github/MarianaAlves01/ProjetoIA/blob/main/ProjetoFinal_MotivosEndividamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Importar bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Carregar dados
data = pd.read_csv('train.csv', low_memory=False)

# Tratamento de valores ausentes
# Preenchendo valores categóricos ausentes com 'Desconhecido'
categorical_cols_to_fill = ['Type_of_Loan', 'Credit_Mix', 'Payment_Behaviour']
for col in categorical_cols_to_fill:
    data[col] = data[col].fillna('Desconhecido')

# Convertendo colunas problemáticas em numéricas antes de preencher valores ausentes
numerical_cols_to_fill = ['Monthly_Inhand_Salary', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries',
                          'Amount_invested_monthly', 'Monthly_Balance']
for col in numerical_cols_to_fill:
    data[col] = pd.to_numeric(data[col], errors='coerce')  # Converter valores para numérico
    data[col] = data[col].fillna(data[col].median())  # Preencher valores ausentes com a mediana

# Removendo colunas irrelevantes ou com muitos valores ausentes
cols_to_drop = ['Name', 'SSN', 'ID', 'Customer_ID']
data = data.drop(columns=cols_to_drop)

# Convertendo colunas categóricas em numéricas
label_encoders = {}
categorical_cols = data.select_dtypes(include='object').columns
for col in categorical_cols:
    if col != 'Credit_Score':  # Excluir a variável alvo
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        label_encoders[col] = le

# Dividindo dados em recursos e alvo
X = data.drop(columns=['Credit_Score'])
y = data['Credit_Score']

# Dividindo em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinando modelo de Machine Learning
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Fazendo previsões e avaliando o modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Importância das características
feature_importances = pd.DataFrame({
    'Característica': X.columns,
    'Importância': model.feature_importances_
}).sort_values(by='Importância', ascending=False)

# Criando planilha com resultados
output_data = {
    'Acurácia': [accuracy],
    'Importância das Características': feature_importances.to_dict(orient='records')
}

# Salvando resultados em um arquivo Excel
with pd.ExcelWriter('resultados_modelo.xlsx') as writer:
    pd.DataFrame({'Acurácia': [accuracy]}).to_excel(writer, sheet_name='Acurácia do Modelo', index=False)
    feature_importances.to_excel(writer, sheet_name='Importância das Características', index=False)

print("Resultados exportados para 'resultados_modelo.xlsx'")


Resultados exportados para 'resultados_modelo.xlsx'
